# This notebook is demonstrating (and testing) scenarios for generating requests to the AICI server (evolving API)

In [1]:
import sys
#sys.path.insert(0, 'c:\\users\\emrek\\source\\guidance\\prompt-plan\\promptlib\\')
sys.path.insert(0, '/workspaces/aici/promptlib')


In [2]:
from promptlib import PromptNode
from promptlib import set_model, append, gen, choose, begin, end
from promptlib.models import LLM, TransformersLLM
from promptlib.endpoints import AICI

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Here are examples of plans generated by promptlib

First, we show building a single path through a prompt tree.

In [3]:
llm = TransformersLLM("gpt2")

pn = PromptNode().set_model(llm)                                          \
    .append("Here is a generated prompt:", tag="prompt1").gen(max_tokens=10)             \
    .append("Here is another generated prompt. ").gen(max_tokens=10, ignore=["prompt1"])

aici_steps = pn.build_linear_plan()
print(aici_steps)

{'steps': [{'Model': {'model': 'gpt2'}}, {'Fixed': {'text': 'Here is a generated prompt:', 'tag': 'prompt1'}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}}}, {'Fixed': {'text': 'Here is another generated prompt. '}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}, 'mask_tags': ['prompt1']}}]}


In [4]:
llm = TransformersLLM("gpt2")

pn0  = PromptNode()

pn = pn0.set_model(llm)                                          \
    .begin(id="block1") \
    .append("Here is a generated prompt. Resp:").gen(max_tokens=10)             \
    .append("Here is another generated prompt. Resp:").gen(max_tokens=10)      \
    .end() \
    .append("Here is a final prompt. Resp:").gen(max_tokens=10)  
    

aici_steps = pn0.build_tree_plan()
print(aici_steps)

{'steps': [{'Model': {'model': 'gpt2'}}, {'block': {'steps': [{'Fixed': {'text': 'Here is a generated prompt. Resp:'}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}}}, {'Fixed': {'text': 'Here is another generated prompt. Resp:'}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}}}], 'id': 'block1'}}, {'Fixed': {'text': 'Here is a final prompt. Resp:'}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}}}]}


The constraints we are imposing here have both a python code implementation and also return a regex that can impose the same constraint.

The general idea is that a constraint in python has to match some implementation of a constraint in the Rust/WASM AST-Runner.  If none exists, then we need to upload imperative code (in WASM) to the server to implement the constraint.

In [5]:
llm = TransformersLLM("gpt2")

from promptlib.constraints import DummyCharacterConstraint, OddEvenDigitConstraint

pn0  = PromptNode()

pn = pn0.set_model(llm)                                          \
    .begin(id="block1") \
    .constrain(DummyCharacterConstraint(['h','e','l','o'])) \
    .constrain(OddEvenDigitConstraint()) \
    .append("Here is a generated prompt. Resp:").gen(max_tokens=10)             \
    .append("Here is another generated prompt. Resp:").gen(max_tokens=10)      \
    .end() \
    .append("Here is a final prompt. Resp:").gen(max_tokens=10)  
    

aici_steps = pn0.build_tree_plan()
print(aici_steps)

{'steps': [{'Model': {'model': 'gpt2'}}, {'block': {'steps': [{'constraint': {'type': 'regex', 'rx': '[h|e|l|o]+'}}, {'constraint': {'type': 'regex', 'rx': '^(?:[13579][02468])*[13579]|[02468]([13579]?|(?:[13579][02468])*)$'}}, {'Fixed': {'text': 'Here is a generated prompt. Resp:'}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}}}, {'Fixed': {'text': 'Here is another generated prompt. Resp:'}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}}}], 'id': 'block1'}}, {'Fixed': {'text': 'Here is a final prompt. Resp:'}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}}}]}


Here's an example of actually uploading a program and plan to the server

In [3]:
ep = AICI(base_url="http://127.0.0.1:8080/v1/", wasm_runner_path="/workspaces/aici/aici_ast_runner/target/opt.wasm")

endpoint = PromptNode().set_endpoint(ep)

pn = endpoint.append("Please answer the following questions:\n Q: Who is the president of the USA?\n A:").gen(max_tokens=10)             \
    .append("\nQ: And who is the vice president?\n A:", attrs=["allcaps"]).gen(max_tokens=20) \
    .append("\nPlease give a url with evidence\n http://").gen(max_tokens=20)

endpoint.run(pn)

upload module... 3270kB -> 10168kB id:292267c9
Please answer the following questions:
 Q: Who is the president of the USA?
 A: The president of the USA is Donald Trump.
Q: And who is the vice president?
 A: The vice president of the USA is Mike Pence.
Please give a url with evidence
 http://www.whitehouse.gov/
http://www.whitehouse.gov/president/


(['Please answer the following questions:\n Q: Who is the president of the USA?\n A: The president of the USA is Donald Trump.\nQ: And who is the vice president?\n A: The vice president of the USA is Mike Pence.\nPlease give a url with evidence\n http://www.whitehouse.gov/\nhttp://www.whitehouse.gov/president/'],
 {'model': '',
  'prompt': '',
  'max_tokens': 200,
  'n': 1,
  'temperature': 0,
  'stream': True,
  'aici_module': '292267c9153c77780d465413dcaae79a5f0f4f37736af247f906381c368d8cfe',
  'aici_arg': {'steps': [{'Fixed': {'text': {'String': {'str': 'Please answer the following questions:\n Q: Who is the president of the USA?\n A:'}},
      'tag': '_2',
      'label': '_2'}},
    {'Gen': {'max_tokens': 10, 'tag': '_2', 'label': '_2'}},
    {'Fixed': {'text': {'String': {'str': '\nQ: And who is the vice president?\n A:'}},
      'tag': '_3',
      'label': '_3'}},
    {'Gen': {'max_tokens': 20, 'tag': '_3', 'label': '_3'}},
    {'Fixed': {'text': {'String': {'str': '\nPlease give

In [3]:
ep = AICI(base_url="http://127.0.0.1:8080/v1/", wasm_runner_path="/workspaces/aici/aici_ast_runner/target/opt.wasm")

endpoint = PromptNode().set_endpoint(ep)

pn = endpoint.append("Please answer the following questions:\n Q: Who is the president of the USA?\n A:").gen(max_tokens=10)             \
    .append("\nQ: And who is the vice president?\n A:", attrs=["allcaps"]).gen(max_tokens=20) \
    .append("\nPlease give a url with evidence\n http://").gen(max_tokens=20, ignore=["allcaps"])

steps = endpoint.build_tree_plan()
print(steps)

endpoint.run(pn)

upload module... 3270kB -> 10168kB id:292267c9
{'steps': [{'Fixed': {'text': {'String': {'str': 'Please answer the following questions:\n Q: Who is the president of the USA?\n A:'}}, 'tag': '_2', 'label': '_2'}}, {'Gen': {'max_tokens': 10, 'tag': '_2', 'label': '_2'}}, {'Fixed': {'text': {'String': {'str': '\nQ: And who is the vice president?\n A:'}}, 'tag': '_3', 'label': '_3'}}, {'Gen': {'max_tokens': 20, 'tag': '_3', 'label': '_3'}}, {'Fixed': {'text': {'String': {'str': '\nPlease give a url with evidence\n http://'}}, 'tag': '_4', 'label': '_4'}}, {'Fork': {'branches': [[{'Fixed': {'following': '_3', 'text': {'Concat': {'parts': [{'String': {'str': '{{_3}}'}}, {'String': {'str': '\nPlease give a url with evidence\n http://'}}]}}}}, {'Gen': {'max_tokens': 20, 'tag': '_4', 'label': '_4', 'stmts': [{'Set': {'var': 'gen_var_4', 'expr': {'Current': {}}}}]}}], [{'Wait': {'vars': ['gen_var_4']}}, {'Fixed': {'text': {'Var': {'var': 'gen_var_4'}}}}]]}}]}


ChunkedEncodingError: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))

In [6]:
ep = AICI(base_url="http://127.0.0.1:8080/v1/", wasm_runner_path="/workspaces/aici/aici_ast_runner/target/opt.wasm")

endpoint = PromptNode().set_endpoint(ep)

pn = endpoint.append(" Please answer the following questions") \
    .append("(And answer in ALL CAPS):", tag="allcaps") \
    .append("(and answer in all lowercase)", tag="alllower") \
    .append("\nQ: Why did Einstein say he liked to play violin?\n A:").gen(max_tokens=30, ignore=["allcaps"])             \
    .append("\nQ: Who is the president of the USA?\n A:").gen(max_tokens=30, ignore=["alllower"])             \
    .append("\nQ: And who is the vice president?\n A:").gen(max_tokens=20) \
    .append("\nPlease give a url with evidence\n http://").gen(max_tokens=20)

aici_steps = endpoint.build_tree_plan()
print(aici_steps)

endpoint.run(pn)

upload module... 3185kB -> 9853kB id:f46c460b
{'steps': [{'Fixed': {'text': ' Please answer the following questions'}}, {'Fixed': {'text': '(And answer in ALL CAPS):', 'tag': 'allcaps'}}, {'Fixed': {'text': '(and answer in all lowercase)', 'tag': 'alllower'}}, {'Fixed': {'text': '\nQ: Why did Einstein say he liked to play violin?\n A:'}}, {'Gen': {'max_tokens': 30, 'genargs': {}, 'mask_tags': ['allcaps']}}, {'Fixed': {'text': '\nQ: Who is the president of the USA?\n A:'}}, {'Gen': {'max_tokens': 30, 'genargs': {}, 'mask_tags': ['alllower']}}, {'Fixed': {'text': '\nQ: And who is the vice president?\n A:'}}, {'Gen': {'max_tokens': 20, 'genargs': {}}}, {'Fixed': {'text': '\nPlease give a url with evidence\n http://'}}, {'Gen': {'max_tokens': 20, 'genargs': {}}}]}
Please answer the following questions(And answer in ALL CAPS):(and answer in all lowercase)
Q: Why did Einstein say he liked to play violin?
 A: Because he was a little out of tune.
Q: Who is the president of the USA?
 A: I don't

In [ ]:
ep = AICI(base_url="http://127.0.0.1:8080/v1/", wasm_runner_path="/workspaces/aici/aici_ast_runner/target/opt.wasm")

endpoint = PromptNode().set_endpoint(ep)

pn = endpoint.append(" Please answer the following questions") \
    .append("(And answer in ALL CAPS):", tag="allcaps") \
    .append("\nQ: Why did Einstein say he liked to play violin?\n A:").gen(max_tokens=30)             \
    .append("\nQ: Who is the president of the USA?\n A:").gen(max_tokens=30, ignore=["allcaps"])             \
    .append("\nQ: And who is the vice president?\n A:").gen(max_tokens=20) \
    .append("\nPlease give a url with evidence\n http://").gen(max_tokens=20)

aici_steps = endpoint.build_tree_plan()
print(aici_steps)

endpoint.run(pn)

In [30]:
ep = AICI(base_url="http://127.0.0.1:8080/v1/", wasm_runner_path="/workspaces/aici/aici_ast_runner/target/opt.wasm")

endpoint = PromptNode().set_endpoint(ep)

pn = endpoint.append("Please answer the following questions") \
    .append("\nQ: Why did Einstein say he liked to play violin?\n") \
    .append("\n(And always answer in ALL CAPS)", tag="allcaps") \
    .append("\nA:") \
    .gen(max_tokens=30) #, ignore=["allcaps"])

#aici_steps = endpoint.build_tree_plan()
#print(aici_steps)

endpoint.run(pn)

upload module... 2688kB -> 7999kB id:6abb139f
{'model': '', 'prompt': '', 'max_tokens': 200, 'n': 1, 'temperature': 0, 'stream': True, 'aici_module': '6abb139fdd3b596815e342dc3bb124a8cc4fbc560a75f3e94965f0e8954957e1', 'aici_arg': {'steps': [{'Fixed': {'text': 'Please answer the following questions'}}, {'Fixed': {'text': '\nQ: Why did Einstein say he liked to play violin?\n'}}, {'Fixed': {'text': '\n(And always answer in ALL CAPS)', 'tag': 'allcaps'}}, {'Fixed': {'text': '\nA:'}}, {'Gen': {'max_tokens': 30, 'genargs': {}}}]}}
Please answer the following questions
Q: Why did Einstein say he liked to play violin?

(And always answer in ALL CAPS)
A: BECAUSE HE WAS A VIOLINIST

Comment: I'm voting to close this question as off-topic because it is


In [34]:
ep = AICI(base_url="http://127.0.0.1:8080/v1/", wasm_runner_path="/workspaces/aici/aici_ast_runner/target/opt.wasm")

endpoint = PromptNode().set_endpoint(ep)

pn = endpoint.append("Please repeat the following list in order:") \
    .append("\nApple") \
    .append("\nCherries", tag="selected") \
    .append("\nGrapes", tag="selected") \
    .append("\nStrawberries") \
    .append("\nBananas", tag="selected") \
    .append("\n\nOk now please repeat the list:\n") \
    .gen(max_tokens=30, ignore=["selected"])

#aici_steps = endpoint.build_tree_plan()
#print(aici_steps)

endpoint.run(pn)

upload module... 2688kB -> 7999kB id:6abb139f
{'model': '', 'prompt': '', 'max_tokens': 200, 'n': 1, 'temperature': 0, 'stream': True, 'aici_module': '6abb139fdd3b596815e342dc3bb124a8cc4fbc560a75f3e94965f0e8954957e1', 'aici_arg': {'steps': [{'Fixed': {'text': 'Please repeat the following list in order:'}}, {'Fixed': {'text': '\nApple'}}, {'Fixed': {'text': '\nCherries', 'tag': 'selected'}}, {'Fixed': {'text': '\nGrapes', 'tag': 'selected'}}, {'Fixed': {'text': '\nStrawberries'}}, {'Fixed': {'text': '\nBananas', 'tag': 'selected'}}, {'Fixed': {'text': '\n\nOk now please repeat the list:\n'}}, {'Gen': {'max_tokens': 30, 'genargs': {}, 'mask_tags': ['selected']}}]}}
Please repeat the following list in order:
Apple
Cherries
Grapes
Strawberries
Bananas

Ok now please repeat the list:
Apple
Banana
Orange
Pear

Please repeat the following list in order:
Apple
Banana



In [1]:
ep = AICI(base_url="http://127.0.0.1:8080/v1/", wasm_runner_path="/workspaces/aici/aici_ast_runner/target/opt.wasm")

endpoint = PromptNode().set_endpoint(ep)

pn = endpoint.append("[INST] Please repeat the following list in order:") \
    .append("\nApple") \
    .append("\nCherries", attrs=["selected"]) \
    .append("\nGrapes", attrs=["selected"]) \
    .append("\nStrawberries") \
    .append("\nBananas", attrs=["selected"]) \
    .append("\n\nOk now please repeat the list and say DONE when DONE:[/INST]\n") \
    .gen(max_tokens=30, stop_at="DONE", ignore=["selected"])

aici_steps = endpoint.build_tree_plan()
print(aici_steps)

results = endpoint.runAll()

NameError: name 'AICI' is not defined

In [45]:
print(results[0][0])


Please repeat the following list in order:
Apple
Cherries
Grapes
Strawberries
Bananas

Strawberries

Ok now please repeat the list:
Apple
Strawberries

Please repeat the following list in order:
Apple
Strawberries

Ok now please


In [3]:
ep = AICI(base_url="http://127.0.0.1:8080/v1/", wasm_runner_path="/workspaces/aici/aici_ast_runner/target/opt.wasm")


endpoint = PromptNode().set_endpoint(ep)

pn0 = endpoint.append("Please remember the following words: Apple, Bananas, and Telephone\n")

pn1 = pn0.append("\nNow repeat those words in French: ") \
    .gen(max_tokens=30, set_var="french")

pn2 = pn0.append("\nNow repeat those words in German: ") \
    .gen(max_tokens=30, set_var="german")

pn3 = pn0.wait(["french", "german"]) \
    .append("\nIn german {{german}} means {{french}} in french")

aici_steps = endpoint.build_tree_plan()
print(aici_steps)

endpoint.runAll()

upload module... 3185kB -> 9853kB id:f46c460b
{'steps': [{'Fixed': {'text': 'Please remember the following words: Apple, Bananas, and Telephone\n', 'expand_vars': True, 'tag': '_2', 'label': '_2'}}, {'Fork': {'branches': [[{'Fixed': {'text': '\nNow repeat those words in French: ', 'expand_vars': True, 'tag': '_3', 'label': '_3'}}, {'Gen': {'max_tokens': 30, 'tag': '_2', 'label': '_2', 'set_var': 'french'}}], [{'Fixed': {'text': '\nNow repeat those words in German: ', 'expand_vars': True, 'tag': '_4', 'label': '_4'}}, {'Gen': {'max_tokens': 30, 'tag': '_3', 'label': '_3', 'set_var': 'german'}}], [{'Wait': {'vars': ['french', 'german'], 'tag': '_2', 'label': '_2'}}, {'Fixed': {'text': '\nIn german {{german}} means {{french}} in french', 'expand_vars': True, 'tag': '_5', 'label': '_5'}}]]}}]}
['Please remember the following words: Apple, Bananas, and Telephone\n\nNow repeat those words in French: \n\nApple: Pomme\nBananas: Bananes\nTelephone: Téléphone\n\nNow repeat those words in Spanish

(['Please remember the following words: Apple, Bananas, and Telephone\n\nNow repeat those words in French: \n\nApple: Pomme\nBananas: Bananes\nTelephone: Téléphone\n\nNow repeat those words in Spanish',
  "Please remember the following words: Apple, Bananas, and Telephone\n\nNow repeat those words in German: \n\nApple: Apfel\nBananas: Bananen\nTelephone: Telefon\n\nNow, let's try",
  "Please remember the following words: Apple, Bananas, and Telephone\n░\nIn german \n\nApple: Apfel\nBananas: Bananen\nTelephone: Telefon\n\nNow, let's try means \n\nApple: Pomme\nBananas: Bananes\nTelephone: Téléphone\n\nNow repeat those words in Spanish in french"],
 {'model': '',
  'prompt': '',
  'max_tokens': 200,
  'n': 1,
  'temperature': 0,
  'stream': True,
  'aici_module': 'f46c460b5c4a5de00e4d26ad04a421136d2b4257334b046a826d1e1dccad5236',
  'aici_arg': {'steps': [{'Fixed': {'text': 'Please remember the following words: Apple, Bananas, and Telephone\n',
      'expand_vars': True,
      'tag': '_2'

In [ ]:
ep = AICI(base_url="http://127.0.0.1:8080/v1/", wasm_runner_path="/workspaces/aici/aici_ast_runner/target/opt.wasm")


endpoint = PromptNode().set_endpoint(ep)

pn0 = endpoint.append("Please remember the following words: Apple, Bananas, and Telephone\n")

pn1 = pn0.append("\nNow repeat those words in French: ") \
    .gen(max_tokens=30, set_var="french")

pn2 = pn0.append("\nNow repeat those words in German: ") \
    .gen(max_tokens=30, set_var="german")

pn3 = pn0.wait(["french", "german"]) \
    .append("\nIn german {{german}} means {{french}} in french")

aici_steps = endpoint.build_tree_plan()
print(aici_steps)

endpoint.runAll()

In [ ]:
pn0 = endpoint.append("Please remember the following words: Apple, Bananas, and Telephone\n")

pn1 = pn0.append("\nNow repeat those words in French: ") \
    .gen(max_tokens=30, set_var="french")

pn2 = pn1.append("\nNow repeat those words in German: ", following=pn0.id) \
    .gen(max_tokens=30, set_var="german")

pn3 = pn2.append("\nIn german {{german}} means {{french}} in french", following=pn0.id)


In [3]:
ep = AICI(base_url="http://127.0.0.1:8080/v1/", wasm_runner_path="/workspaces/aici/aici_ast_runner/target/opt.wasm")

endpoint = PromptNode().set_endpoint(ep)

pn = endpoint.append("Please remember the following items:") \
    .append("\nApple") \
    .append("\nCherries") \
    .append("\nTomatoes", attrs=["veg"]) \
    .append("\nCucumber", attrs=["veg"]) \
    .append("\nStrawberries") \
    .append("\nPears") \
    .append("\nSquash", attrs=["veg"]) \
    .append("\n\nOk now please repeat the list:\n") \
    .gen(max_tokens=30, stop_at="\n\n", ignore=["veg"]) # Ignore fruits that pretend to be vegetables

aici_steps = endpoint.build_tree_plan()
print(aici_steps)

result = endpoint.run(pn)

upload module... 3185kB -> 9853kB id:f46c460b
{'steps': [{'Fixed': {'text': 'Please remember the following items:', 'expand_vars': True, 'tag': '_2', 'label': '_2'}}, {'Fixed': {'text': '\nApple', 'expand_vars': True, 'tag': '_3', 'label': '_3'}}, {'Fixed': {'text': '\nCherries', 'expand_vars': True, 'tag': '_4', 'label': '_4'}}, {'Fixed': {'text': '\nTomatoes', 'expand_vars': True, 'tag': '_5', 'label': '_5'}}, {'Fixed': {'text': '\nCucumber', 'expand_vars': True, 'tag': '_6', 'label': '_6'}}, {'Fixed': {'text': '\nStrawberries', 'expand_vars': True, 'tag': '_7', 'label': '_7'}}, {'Fixed': {'text': '\nPears', 'expand_vars': True, 'tag': '_8', 'label': '_8'}}, {'Fixed': {'text': '\nSquash', 'expand_vars': True, 'tag': '_9', 'label': '_9'}}, {'Fixed': {'text': '\n\nOk now please repeat the list:\n', 'expand_vars': True, 'tag': '_10', 'label': '_10'}}, {'Fork': {'branches': [[{'Fixed': {'following': '_5', 'text': '\nStrawberries\nPears\n\nOk now please repeat the list:\n', 'expand_vars':

ChunkedEncodingError: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))

In [23]:
print(result[0][0])

Please remember the following items:
Apple
Cherries
Tomatoes
Cucumber
Strawberries
Pears
Squash

Ok now please repeat the list:

Strawberries
Pears

Ok now please repeat the list:




In [24]:
result[1]

{'model': '',
 'prompt': '',
 'max_tokens': 200,
 'n': 1,
 'temperature': 0,
 'stream': True,
 'aici_module': 'f46c460b5c4a5de00e4d26ad04a421136d2b4257334b046a826d1e1dccad5236',
 'aici_arg': {'steps': [{'Fixed': {'text': 'Please remember the following items:',
     'expand_vars': True,
     'tag': '_66',
     'label': '_66'}},
   {'Fixed': {'text': '\nApple',
     'expand_vars': True,
     'tag': '_67',
     'label': '_67'}},
   {'Fixed': {'text': '\nCherries',
     'expand_vars': True,
     'tag': '_68',
     'label': '_68'}},
   {'Fixed': {'text': '\nTomatoes',
     'expand_vars': True,
     'tag': '_69',
     'label': '_69'}},
   {'Fixed': {'text': '\nCucumber',
     'expand_vars': True,
     'tag': '_70',
     'label': '_70'}},
   {'Fixed': {'text': '\nStrawberries',
     'expand_vars': True,
     'tag': '_71',
     'label': '_71'}},
   {'Fixed': {'text': '\nPears',
     'expand_vars': True,
     'tag': '_72',
     'label': '_72'}},
   {'Fixed': {'text': '\nSquash',
     'expand_va